# Оценки уровня убийств (на 100 000 населения)

In [1]:
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке.

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/estimates-of-rates-of-homicides-per-100-000-population

Информация есть только за 2000 - 2019 года.\
За 2020 г. нет данных.\
Заполним  данные для 2020 года средним за последние 3 года по стране.

In [2]:
who_homicides_data = pd.read_csv('../data/who_homicides.csv')

who_homicides_data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,VIOLENCE_HOMICIDERATE,Estimates of rates of homicides per 100 000 po...,numeric,WPR,Western Pacific,Country,JPN,Japan,Year,2019,...,NaN,NaN,0.19,NaN,0.23,0.2 [0.2-0.2],NaN,NaN,EN,2021-02-09 00:00:00
1,VIOLENCE_HOMICIDERATE,Estimates of rates of homicides per 100 000 po...,numeric,WPR,Western Pacific,Country,SGP,Singapore,Year,2019,...,NaN,NaN,0.20,NaN,0.32,0.3 [0.2-0.3],NaN,NaN,EN,2021-02-09 00:00:00


Оставим только те поля, по которым будем объединять таблицу с основной.

- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - оценка уровня убийств (на 100 000 населения)

In [3]:
who_homicides_data = who_homicides_data[[
    'SpatialDimValueCode', 'Period', 'FactValueNumeric'
]]

Переименуем столбец `FactValueNumeric` в `Homicides`.

In [4]:
who_homicides_data.rename(
    columns={'FactValueNumeric': 'Homicides'},
    inplace=True,
)
who_homicides_data.head()

,SpatialDimValueCode,Period,Homicides
0,JPN,2019,0.21
1,SGP,2019,0.25
2,BHR,2019,0.31
3,QAT,2019,0.50
4,CHE,2019,0.49


Посмотрим на пропуски.

In [5]:
who_homicides_data.isna().sum()

SpatialDimValueCode    0
Period                 0
Homicides              0
dtype: int64

Пропусков нет.

Составим таблицу с данными для 2020 г.

In [6]:
# Составим пустую таблицу
who_homicides_2020_data = pd.DataFrame(columns=who_homicides_data.columns)

who_homicides_2020_data

,SpatialDimValueCode,Period,Homicides


In [7]:
# Составим список из кодов стран
location_codes = list(who_homicides_data['SpatialDimValueCode'].unique())

# И заполним ими поле с кодом страны
who_homicides_2020_data['SpatialDimValueCode'] = location_codes

# Заполним поле с годом
who_homicides_2020_data['Period'] = 2020

# Посмотрим на результат
who_homicides_2020_data.head()

,SpatialDimValueCode,Period,Homicides
0,JPN,2020,NaN
1,SGP,2020,NaN
2,BHR,2020,NaN
3,QAT,2020,NaN
4,CHE,2020,NaN


Теперь заполним поле `Homicides` средним по последним 3-м годам соответствующей страны.

In [8]:
# Маска по последним 3-м годам существующей таблицы
mask = who_homicides_data['Period'].isin([2019, 2018, 2017])

# Посчитаем среднее за 3 года по каждой стране
who_homicides_mean_last_3_year_data = \
    who_homicides_data[mask].groupby('SpatialDimValueCode').agg(
        HomicidesMean=('Homicides', 'mean')
    )
who_homicides_mean_last_3_year_data.head()

,HomicidesMean
SpatialDimValueCode,
AFG,8.443333
AGO,9.376667
ALB,3.543333
ARE,0.736667
ARG,6.163333


In [9]:
# Заполним полученными средними столбец Homicides для 2020 г.
who_homicides_2020_data['Homicides'] = \
    who_homicides_2020_data['SpatialDimValueCode'].apply(
        lambda code: who_homicides_mean_last_3_year_data.loc[code]['HomicidesMean']
    )

who_homicides_2020_data.head()

,SpatialDimValueCode,Period,Homicides
0,JPN,2020,0.220000
1,SGP,2020,0.273333
2,BHR,2020,0.290000
3,QAT,2020,0.533333
4,CHE,2020,0.520000


Убедимся, что нет пропусков.

In [10]:
who_homicides_2020_data.isna().sum()

SpatialDimValueCode    0
Period                 0
Homicides              0
dtype: int64

In [11]:
# Присоединим таблицу к предыдущей
who_homicides_data = pd.concat([
    who_homicides_data,
    who_homicides_2020_data,
], ignore_index=True)

who_homicides_data.head()

,SpatialDimValueCode,Period,Homicides
0,JPN,2019,0.21
1,SGP,2019,0.25
2,BHR,2019,0.31
3,QAT,2019,0.50
4,CHE,2019,0.49


In [12]:
# Сохраним полученную таблицу
who_homicides_data.to_csv('../data/who_homicides_prepared.csv', index=False)